In [1]:
import sys
import numpy as np

import torch.nn as nn
import torch

import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

sys.path.append('../')
import utils.DataGen as DataGen

In [ ]:
xmin = -8#param {type:"number"}
xmax = 8#param {type:"number"}
N=15#param {type:"slider", min:5, max:30, step:1}
n_state=0#param {type:"slider", min:0, max:10, step:1}
n_points = 100#param {type:"integer"}
n_samples=2000#param {type:"integer"}
#markdown ---

# We generate the values of alpha
alpha_min = np.array([-150,-10,0.3,-0.1,0])/50
alpha_max = np.array([50,5,1.0,0.1,0.25])/50

k = alpha_min.shape[0]

r_alpha = np.random.random((int(n_samples*0.8), k)) # Values between 0 and 1
alpha1 = r_alpha*(alpha_max - alpha_min)+ alpha_min # random alpha

alpha_min2 = np.array([-10,-1,0.1,0,0])
alpha_max2 = np.array([5,1,1.0,0,0])

r_alpha = np.random.random((int(n_samples*0.2), k)) # Values between 0 and 1
alpha2 = r_alpha*(alpha_max2 - alpha_min2)+ alpha_min2 # random alpha
alpha = np.concatenate((alpha1, alpha2))

data_gen = DataGen.potencial_datos(alpha_min, alpha_max, N)

# Generate the energies, wavefunctions and potentials
E, a, alpha = data_gen.generate_data(n_samples, alpha)
waves, x,phis = data_gen.final_wavefunction( xmin, xmax, n_points, a)
V, _ = data_gen.evaluar_potencial( xmin, xmax, n_points, alpha)
idx=-1

In [5]:
V = np.loadtxt('../in/V.txt', dtype=float)
waves = np.loadtxt('../in/waves.txt', dtype=float)
print(f'V.shape = {V.shape}')
print(f'waves.shape = {waves.shape}')

V.shape = (2000, 100)
waves.shape = (2000, 100)


In [17]:
from sklearn.model_selection import train_test_split

# Split train and test 
idx_train, idx_test, wave_train, wave_test = train_test_split(np.arange(V.shape[0]),
                                                              waves, test_size=0.3, random_state=123)
V_train = V[idx_train,:]
V_test = V[idx_test,:]
print(f'V_train.shape = {V_train.shape}')
print(f'V_test.shape = {V_test.shape}')

V_train.shape = (1400, 100)
V_test.shape = (600, 100)


In [9]:
model = nn.Sequential(
    nn.Linear(in_features = n_points, out_features= 256),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(128, n_points),
)

In [18]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
n_points = 100

epochs = 10000

V_train = torch.from_numpy(V_train).float()
wave_train = torch.from_numpy(wave_train).float()

for i in range(epochs):

  y = model(V_train)

  optimizer.zero_grad()

  l = loss(y, wave_train)

  l.backward()

  optimizer.step()

  if i%100 == 0:
    print(f'Epoch {i}/{epochs} Loss {losss}')

In [19]:
model = torch.load('../Models/model_adam.pt')
model.eval()

Sequential(
  (0): Linear(in_features=100, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.1, inplace=False)
  (6): Linear(in_features=256, out_features=128, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.1, inplace=False)
  (9): Linear(in_features=128, out_features=128, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.1, inplace=False)
  (12): Linear(in_features=128, out_features=100, bias=True)
)

In [21]:
V_test = torch.from_numpy(V_test).float()
wave_test = torch.from_numpy(wave_test).float()

predict = model(V_test)

print("Test MSE: %f" % loss(predict, wave_test))

Test MSE: 0.000032


In [22]:
xmin = -8
xmax = 8

predict = predict.detach().numpy()
x = np.arange(xmin, xmax, (xmax - xmin)/n_points)

In [24]:
idx = 2
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=predict[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=wave_test[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=V_test[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Random potential prediction",
    xaxis_title="x")

fig.show()